Projeto 3 - Samara e Tainara

Git Hub: www.github.com/tainarasm




In [9]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os


In [10]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\saah_\Desktop\INSPER\Ciência dos Dados\Projeto 3


In [14]:
dados_aviao = pd.read_excel('aircraft.xlsx')
dados_acidentes = pd.read_excel('occurrences.xlsx')